# Titre

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import requests
import socket
from plotly.subplots import make_subplots

In [29]:
path_blockchain_actor = "../../timeseries/blockchain_by_actor.csv" 
path_external = "../../timeseries/external.csv" 
path_global = "../../timeseries/global.csv"

In [30]:
df_actor = pd.read_csv(path_blockchain_actor, index_col=0)
df_actor.head(5)

,year,month,day,identity,received,nb_received,date,sum_fee,mean_fee_for100,nb_transactions,sent,self_spent,self_spent_estimated,nb_spent,spent
0,2015.0,1.0,1.0,HappyCoins.com,1.377590e+09,4.0,2015-01-01,49596.0,0.004957,4.0,1.277500e+09,0.0,0.0,8.0,1.277500e+09
1,2015.0,1.0,1.0,821,2.476057e+09,5.0,2015-01-01,10000.0,0.000851,1.0,1.174552e+09,0.0,0.0,2.0,1.174552e+09
2,2015.0,1.0,1.0,FortuneJack.com,1.201470e+08,29.0,2015-01-01,83000.0,0.194670,11.0,1.541127e+08,14158970.0,14158970.0,22.0,1.399538e+08
3,2015.0,1.0,1.0,396,1.906135e+10,70.0,2015-01-01,540000.0,0.023738,54.0,1.842989e+10,0.0,0.0,108.0,1.842989e+10
4,2015.0,1.0,1.0,SatoshiMines.com,7.401319e+08,193.0,2015-01-01,740000.0,0.447934,45.0,1.813624e+09,35044568.0,35044568.0,84.0,1.778579e+09


This file contains time series describing the 100 actors with the highest activity (defined in terms of the number of active days) over the period.

- **identity**: Identifier of the actor, which can be a name or a unique number
- **received**: Total amount received
- **spent**: Total amount spent
- **nb_received**: Number of transaction outputs received by the actor
- **nb_transactions**: Number of transactions made by the actor
- **nb_spent**: Number of payments made by the actor (1 transaction = 1 or several payments).
- **sum_fee**: Total transaction fees paid by the actor for transactions for which he is the source
- **mean_fee_for100**: Average fees paid per transaction
- **self_spent**: Amounts observed as sent from the actor to himself
- **self_spent_estimated**: Amounts estimated as likely sent from the actor to himself, but to addresses we do not know. This value is necessarily higher than self_spent.

In [31]:
# List some actors
identity = np.array(df_actor['identity'].unique())
identity[['.' in x for x in identity]]

array(['HappyCoins.com', 'FortuneJack.com', 'SatoshiMines.com',
       'Rollin.io', 'SatoshiDice.com', 'Loanbase.com', 'Bitstamp.net',
       'Paymium.com', 'YoBit.net', 'FYBSG.com', 'Poloniex.com',
       'Bitbond.com', 'ePay.info_CoinJoinMess', 'HolyTransaction.com',
       'SlushPool.com', 'BTCJam.com', 'Matbea.com', 'TheRockTrading.com',
       'Huobi.com', 'HaoBTC.com', 'SafeDice.com', 'CoinSpot.com.au',
       'CoinMotion.com', 'Xapo.com', 'CoinGaming.io', 'OKCoin.com',
       'BitZillions.com', 'SecondsTrade.com', '999Dice.com', 'BX.in.th',
       'SimpleCoin.cz', 'BTC-e.com', 'PocketDice.io', 'BitBargain.co.uk',
       'Bter.com', 'BtcTrade.com', 'Cryptonator.com', 'BTCC.com',
       'CoinPayments.net', 'CoinCafe.com',
       'CoinTrader.net_LocalBitcoins.com', 'LuckyB.it',
       'MercadoBitcoin.com.br', 'Bitcoin.de', 'CoinJar.com', 'Bit-x.com',
       'Vaultoro.com'], dtype=object)

### Cleaning our data

In [32]:
df_actor.dtypes

year                    float64
month                   float64
day                     float64
identity                 object
received                float64
nb_received             float64
date                     object
sum_fee                 float64
mean_fee_for100         float64
nb_transactions         float64
sent                    float64
self_spent              float64
self_spent_estimated    float64
nb_spent                float64
spent                   float64
dtype: object

In [33]:
# convert year, month and day to int
df_actor['year'] = df_actor['year'].astype(int)
df_actor['month'] = df_actor['month'].astype(int)
df_actor['day'] = df_actor['day'].astype(int)

# convert the date to datetime in year-month-day format
df_actor['date'] = pd.to_datetime(df_actor['date'], format='%Y-%m-%d')

# convert identity to string
df_actor['identity'] = df_actor['identity'].astype(str)

# convert nb_transactions, nb_received, nb_spent to int
df_actor['nb_transactions'] = df_actor['nb_transactions'].astype(int)
df_actor['nb_received'] = df_actor['nb_received'].astype(int)
df_actor['nb_spent'] = df_actor['nb_spent'].astype(int)

In [34]:
# get the year, month and day from the date
df_actor['year'] = df_actor['date'].dt.year
df_actor['month'] = df_actor['date'].dt.month
df_actor['day'] = df_actor['date'].dt.day

In [35]:
# save the df to a csv file
df_actor.to_csv('../../timeseries/blockchain_by_actor_cleaned.csv')

### Biggest receivers actors

In [36]:
# Group by identity and sum the received and spent amounts
df_amount = df_actor[['identity', 'received', 'spent']].groupby('identity').sum()
df_amount

,received,spent
identity,,
0,2.073722e+14,1.364461e+14
101,5.399256e+12,5.214319e+12
107,2.756361e+14,1.366241e+14
1070,8.005035e+11,7.886356e+11
1138,6.114027e+11,6.102002e+11
...,...,...
TheRockTrading.com,4.920981e+12,3.983099e+12
Vaultoro.com,1.524106e+12,1.303786e+12
Xapo.com,2.360818e+14,1.381389e+14


In [37]:
# Sort the actors by the amount received

actors_received = df_amount.sort_values(by='received', ascending=False)
actors_received.index[:5]

Index(['Huobi.com', 'ePay.info_CoinJoinMess', 'Bitstamp.net', 'BTC-e.com',
       'OKCoin.com'],
      dtype='object', name='identity')

In [39]:
# Sort the actors by the amount spent

df_amount2 = df_actor[['identity', 'received', 'spent']].groupby('identity').mean()
actors_received_mean = df_amount2.sort_values(by='received', ascending=False)
most_receiving_actor = actors_received_mean.index[:5]
most_receiving_actor

Index(['Huobi.com', 'ePay.info_CoinJoinMess', 'Bitstamp.net', 'BTC-e.com',
       'OKCoin.com'],
      dtype='object', name='identity')

In [55]:
fig = px.bar(actors_received, x=actors_received.index,
             y=actors_received['received'].values,  
             title='Biggest receivers in the bitcoin blockchain',
             labels={'x':'Identity', 'y':'Received'},
             )
fig.show()

In [48]:
def PlotActors(df, actors, column, title):
    fig = go.Figure()
    for actor in actors[:5]:
        date = df[df['identity'] == actor]['date'].values
        fig.add_trace(go.Scatter(x=date, y=df_actor[df_actor['identity'] == actor][column].rolling(30).mean(), 
                                 mode='lines', line=dict(width=1), name=actor))
        fig.update_layout(xaxis=dict(tickvals=date[::200]))
        fig.update_layout(title=title, yaxis_type="log")
        fig.update_layout(showlegend=True, legend=dict(x=0, y=0, orientation="v"))
    fig.show()

PlotActors(df_actor, most_receiving_actor, 'received', 'Top 5 actors receiving the most bitcoins')

### Most spending actors

In [56]:
# Sort the actors by the amount spent
 
actors_spent = df_amount.sort_values(by='spent', ascending=False)
most_spending_actor = actors_spent.index[:5]
most_spending_actor


Index(['ePay.info_CoinJoinMess', 'Huobi.com', 'Bitstamp.net', 'Poloniex.com',
       'BTCC.com'],
      dtype='object', name='identity')

In [58]:
PlotActors(df_actor, most_spending_actor, 'spent', 'Top 5 actors spending the most bitcoins')

Why Huobi suddenly fall down ? 
We can found that following a 2017 ban on Bitcoin exchanges by the Chinese government, Huobi stopped Bitcoin withdrawals. Huobi China continues to operate as a blockchain consulting and research platform. 
https://en.wikipedia.org/wiki/Huobi 

Maybe it can be interesting to see the evolution of the Chinese actors. Compare the evolution of the Chinese actors with other platforms.

In [65]:
# subplot of 2 pie plot with plotly with the biggest spenders and the biggest receivers

import plotly.graph_objects as go


fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=actors_spent['spent'].index, values=actors_spent['spent'].values, name="Biggest spenders"), 1, 1)
fig.add_trace(go.Pie(labels=actors_received['received'].index, values=actors_received['received'].values, name="Biggest receivers"), 1, 2)
fig.update_traces(textposition='inside')
fig.update_layout(title_text="Biggest spenders and receivers in the bitcoin blockchain")
fig.show()

In [62]:
actors_received

Index(['Huobi.com', 'ePay.info_CoinJoinMess', 'Bitstamp.net', 'BTC-e.com',
       'OKCoin.com'],
      dtype='object', name='identity')

#### What is the relationship between the total transaction fees paid (sum_fee) and the number of transactions performed (nb_transactions) for each actor?

Let's analyze the correlation between these two variables to see if actors who perform more transactions also pay more fees.

In [ ]:
# let's compute the correlation between the sum_fee and nb_transactions
df[['sum_fee', 'nb_transactions']].corr()

,sum_fee,nb_transactions
sum_fee,1.000000,0.409517
nb_transactions,0.409517,1.000000


Une corrélation de 0,4 signifie qu'il y a une relation positive modérée entre deux variables. Lorsque la valeur d'une variable augmente/diminue, la valeur de l'autre variable a tendance à augmenter/diminuer également, mais pas de manière systématique.

On ne peut pas affirmer de ce fait que les acteurs qui font le plus grand nombres de transactions paieront un plus grand nombre de frais.

#### Is there a relationship between the proportion of self-issued transactions and the transaction fees paid by actors? 

Let's check if actors who self-issue a larger proportion of transactions tend to pay different transaction fees than those who primarily send funds to third parties.

In [ ]:
# compute the self spent proportion
df['self_spent_proportion'] = df['self_spent'] / df['sent'] * 100

# display the top 5 identities in terms of self spent proportion
df.groupby('identity')['self_spent_proportion'].mean().sort_values(ascending=False).head(5)


identity
Paymium.com               81.309526
ePay.info_CoinJoinMess    73.112700
Bitbond.com               68.926661
SimpleCoin.cz             68.501003
BitZillions.com           63.335812
Name: self_spent_proportion, dtype: float64

In [ ]:
df_self_spent_prop_sorted = df.groupby('identity')['self_spent_proportion'].mean().sort_values(ascending=False)

In [ ]:
# plot the df_test with a bar plot with plotly
# with the title 'Top 50 identities in terms of self spent proportion'
# with the figsize (15, 10)
# with the x axis title 'identity'
# with the y axis title 'self spent proportion (in %)'

fig = px.bar(df_self_spent_prop_sorted.head(100), title='Top 50 identities in terms of self spent proportion', labels={'index': 'identity', 'value': 'self spent proportion (in %)'}, height=600)
fig.show()


We can observe the nature of the actors in terms of spending. As we can see here, for some actors the proportion of self_spending is very high.

It is difficult to predict what are the intentions of such actors. Some might manage exchange platforms, that opper intern transactions for several reasons.

Others might use mixing techniques to hide their transactions, or to dissimulate the origin of their funds.

In [ ]:
df_mean_fee_per_transaction = df.groupby('identity')['mean_fee_for100'].mean()

df_self_spent_prop_raw = df.groupby('identity')['self_spent_proportion'].mean()

df_mean_fee_per_transaction.corr(df_self_spent_prop_raw)


-0.049798379838131165

So there is no clear linear relationship between these two variables. In other words, the fact that an actor issues a higher or lower proportion of self-transactions does not have a direct and predictable impact on the average fees they pay per transaction.